In [2]:
import importlib
import json
from tools import network_tools as network_tools
from tools import firewall_tools as firewall_tools
from tools import summarizer_tool as st
importlib.reload(network_tools)
importlib.reload(firewall_tools)
importlib.reload(st)

<module 'tools.summarizer_tool' from '/home/federico/Desktop/Thesis/AI_Agent_for_honeypot_operations/HoneypotAgentApp/MultiAgentArchitecture/tools/summarizer_tool.py'>

In [3]:
alerts = await network_tools.get_alerts(time_window=2)
len(alerts.get('security_events', []).get('alerts'))

0

In [10]:
alerts = await network_tools.get_alerts(time_window=5)
alerts

{'security_events': {'alerts': [{'category': 'Potentially Bad Traffic',
    'dest_ip': '172.20.0.2',
    'dest_port': 3306,
    'flow_id': 11791875667577,
    'payload': None,
    'proto': 'TCP',
    'severity': 2,
    'signature': 'ET SCAN Suspicious inbound to mySQL port 3306',
    'src_ip': '192.168.100.2',
    'src_port': 51749,
    'timestamp': '2025-10-17T19:01:28.723641+0200'},
   {'category': 'Potentially Bad Traffic',
    'dest_ip': '172.20.0.2',
    'dest_port': 1433,
    'flow_id': 13493541540386,
    'payload': None,
    'proto': 'TCP',
    'severity': 2,
    'signature': 'ET SCAN Suspicious inbound to MSSQL port 1433',
    'src_ip': '192.168.100.2',
    'src_port': 51749,
    'timestamp': '2025-10-17T19:01:28.724037+0200'},
   {'category': 'Attempted Information Leak',
    'dest_ip': '172.20.0.2',
    'dest_port': 1433,
    'flow_id': 13493541540386,
    'payload': None,
    'proto': 'TCP',
    'severity': 3,
    'signature': 'POSSBL PORT SCAN (NMAP -sS)',
    'src_ip': '1

In [11]:
vulnerable_containers =  network_tools.get_docker_containers()
st.build_security_summary(alerts.get("security_events", []), vulnerable_containers.get("vulnerable_containers"))

{'security_events': [{'ip': '172.20.0.2',
   'service': 'unauthorized-rce-docker-1',
   'compromise_indicators': [{'signature': 'ET INFO POSSIBLE Web Crawl using Curl',
     'service': 'tcp/2375',
     'count': 1,
     'severity': 2,
     'src_ip': '192.168.100.2',
     'src_port': 50114,
     'payload': 'GET /showcase HTTP/1.1\r\nHost: 172.20.0.2:2375\r\nUser-Agent: curl/8.15.0\r\nAccept: */*\r\n\r\n',
     'new': True},
    {'signature': 'GPL WEB_SERVER DELETE attempt',
     'service': 'tcp/2375',
     'count': 1,
     'severity': 2,
     'src_ip': '192.168.100.2',
     'src_port': 50274,
     'payload': 'GET /v1.48/containers/a84698b80cdd18b237cdb0a40b9444f707158bb014bd365dd45a0a9c084d0571/json HTTP/1.1\r\nHost: 172.20.0.2:2375\r\nUser-Agent: docker-sdk-python/7.1.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\nPOST /v1.48/containers/a84698b80cdd18b237cdb0a40b9444f707158bb014bd365dd45a0a9c084d0571/wait HTTP/1.1\r\nHost: 172.20.0.2:2375\r\nUser-Age

In [6]:
network_tools.get_docker_containers()

{'vulnerable_containers': [{'service': 'unauthorized-rce-docker-1',
   'image': 'unauthorized-rce-docker:latest',
   'ports': ['2375/tcp', '2376/tcp'],
   'ip': '172.20.0.2'}]}

In [1]:
print(network_tools.check_services_health())
rules = await firewall_tools.get_firewall_rules()
print(rules['firewall_config']['data']['rules'])

NameError: name 'network_tools' is not defined

In [11]:
await firewall_tools.remove_firewall_rule([1, 2])

INFO:tools.firewall_tools:Removing firewall rules: [1, 2]


{'rules_removed_current_epoch': ['R',
  'E',
  'M',
  'O',
  'V',
  'E',
  'D',
  ' ',
  'r',
  'u',
  'l',
  'e',
  ' ',
  '#',
  '1',
  ':',
  ' ',
  ' ',
  ' ',
  ' ',
  'A',
  'C',
  'C',
  'E',
  'P',
  'T',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  't',
  'c',
  'p',
  ' ',
  ' ',
  '-',
  '-',
  ' ',
  ' ',
  '1',
  '7',
  '2',
  '.',
  '2',
  '0',
  '.',
  '0',
  '.',
  '2',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  '1',
  '9',
  '2',
  '.',
  '1',
  '6',
  '8',
  '.',
  '1',
  '0',
  '0',
  '.',
  '0',
  '/',
  '2',
  '4',
  'R',
  'E',
  'M',
  'O',
  'V',
  'E',
  'D',
  ' ',
  'r',
  'u',
  'l',
  'e',
  ' ',
  '#',
  '2',
  ':',
  ' ',
  ' ',
  ' ',
  ' ',
  'A',
  'C',
  'C',
  'E',
  'P',
  'T',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  't',
  'c',
  'p',
  ' ',
  ' ',
  '-',
  '-',
  ' ',
  ' ',
  '1',
  '9',
  '2',
  '.',
  '1',
  '6',
  '8',
  '.',
  '1',
  '0',
  '0',
  '.',
  '0',
  '/',
  '2',
  '4',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  '

In [15]:
honeypot_ip = '172.20.0.2' 
attacker_ip = '192.168.100.2/24'

await firewall_tools.add_allow_rule(source_ip=attacker_ip, dest_ip=honeypot_ip, port=8080)
await firewall_tools.add_allow_rule(source_ip=honeypot_ip, dest_ip=attacker_ip)

INFO:tools.firewall_tools:Adding allow rule: ALLOW 192.168.100.2/24 -> 172.20.0.2 (tcp)
INFO:tools.firewall_tools:Adding allow rule: ALLOW 172.20.0.2 -> 192.168.100.2/24 (tcp)


{'rules_added_current_epoch': 'ALLOW 172.20.0.2 -> 192.168.100.2/24 (tcp)'}